# NOVELTY RERANKING (GLOBAL GENRE-BASED DISTANCE)

In [1]:
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)
from src.experiments.novelty_sweep import run_novelty_sweep

In [2]:
df_nov = run_novelty_sweep(output_csv="novelty_sweep_genre_offline.csv")
df_nov

[INFO] Novelty sweep on 4000 users (train_in rows = 176308, train_out rows = 4000)
[INFO] Evaluating baseline (lambda_nov = 0)


Sweeping novelty λ: 100%|██████████| 8/8 [00:47<00:00,  5.89s/it]

[INFO] Saved novelty sweep results to: /home/sunaj/Desktop/novelty-aware-recommenders/results/sweeps/novelty_sweep_genre_offline.csv


,ndcg,recall,user_coverage,item_gini,publisher_gini,item_coverage,intra_list_similarity,novelty,ease_lambda_reg,novelty_lambda
0,0.230748,0.41975,1.0,0.804540,0.802155,0.070536,0.456687,0.557470,300,0.00
1,0.165827,0.31275,1.0,0.784548,0.796837,0.102209,0.742240,0.426371,300,-2.00
2,0.177077,0.33100,1.0,0.786574,0.794599,0.098442,0.724135,0.431452,300,-1.50
3,0.190688,0.35400,1.0,0.789566,0.796120,0.092964,0.694407,0.440897,300,-1.00
4,0.231134,0.41975,1.0,0.804535,0.803569,0.073275,0.491911,0.533304,300,-0.05
5,0.228261,0.41600,1.0,0.802283,0.795972,0.068995,0.412338,0.591514,300,0.05
6,0.220888,0.40475,1.0,0.798153,0.791832,0.068139,0.374597,0.623312,300,0.10
7,0.206378,0.38000,1.0,0.790601,0.789508,0.066598,0.313551,0.675101,300,0.20


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("../results/sweeps/novelty_sweep_genre.csv")

# sort by novelty_lambda to get smooth curves
df = df.sort_values("novelty_lambda")

# 1) Lambda vs metrics (one plot with 4 lines)
plt.figure(figsize=(8, 5))

for metric in ["ndcg", "recall", "novelty"]:
    plt.plot(df["novelty_lambda"], df[metric], marker="o", label=metric)

plt.xlabel("novelty_lambda")
plt.ylabel("metric value")
plt.title("Metrics vs novelty_lambda")
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

x = df["novelty"]
y = df["ndcg"]
lam = df["novelty_lambda"]

plt.figure(figsize=(6, 4))
plt.scatter(x, y, label="All configs", alpha=0.6)

# draw Pareto curve
pareto = df.sort_values("novelty")
plt.plot(pareto["novelty"], pareto["ndcg"], "r-o", label="Pareto front")

# annotate each point with λ
for xv, yv, lv in zip(x, y, lam):
    plt.annotate(
        f"{lv:.2f}",
        (xv, yv),
        textcoords="offset points",
        xytext=(3, 3),
        fontsize=8,
    )

plt.grid(alpha=0.3)
plt.xlabel("Novelty")
plt.ylabel("NDCG")
plt.title("Accuracy(ndcg)–novelty tradeoff")
plt.tight_layout()
plt.show()

#### Conclusion
- I fixed the EASE regularization at λ_reg = 300 and reranked its top 100 candidates per user using a global genre-based novelty weight `novelty_lambda`.
- increasing `novelty_lambda` increases novelty metric (from about 0.55 to 0.85) and slightly improves item coverage.
- At the same time, increasing `novelty_lambda` decreases accuracy (NDCG drops from about 0.228 to 0.075).
- The accuracy–novelty Pareto curve is strictly decreasing and smooth, so each value of novelty_lambda corresponds to a different compromise.

# SAVE A NOVELTY-RERANKED SUBMISSION FOR CODABENCH

In [ ]:
from src.data.loader import load_interactions, load_games
from src.models.ease import EASE
from src.novelty.rerank import rerank_with_novelty
from src.pipelines.save import save_submission
import numpy as np
from src.config import LAMBDA_REG

# 1. Load data
train = load_interactions(train=True)
test_in = load_interactions(train=False)
games = load_games()

# 2. Load precomputed genre distance
item_distance = np.load("../data/processed/genre_distance.npy")

# 3. Train EASE with fixed lambda_reg
model = EASE(lambda_reg=float(LAMBDA_REG))

# recommend() already uses a Codabench style mapping
recs_base = model.recommend(train, test_in, top_k=100)

In [ ]:
# 4. Pick a novelty weight, for example, 0.3
NOVELTY_LAMBDA = 0.0

# re-rank scores
recs_novel = rerank_with_novelty(
    recs_base,
    test_in,
    item_distance,
    lambda_val=NOVELTY_LAMBDA,
)

# keep only top-20 per user, sorted by score
recs_novel = (
    recs_novel
    .sort_values(["user_id", "score"], ascending=[True, False])
    .groupby("user_id")
    .head(20)
    .reset_index(drop=True)
)

# drop scores for submission
submission = recs_novel[["user_id", "item_id"]]

# 5. Save for Codabench
save_submission(submission, f"ease_lambda{LAMBDA_REG}_nov_{NOVELTY_LAMBDA}")

# NOVELTY RERANKING (USER-SPECIFIC GENRE DISTANCE)

In [ ]:
from src.experiments.user_specific_sweep import run_user_specific_sweep

# Profile-size-based weighting
metrics_profile = run_user_specific_sweep(grouping="profile")
metrics_profile

In [ ]:
# Genre-diversity-based weighting
metrics_genre = run_user_specific_sweep(grouping="genre")
metrics_genre

In [ ]:
# Combined strategy
metrics_combined = run_user_specific_sweep(grouping="combined")
metrics_combined

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

global_df = pd.read_csv("../results/sweeps/novelty_sweep_genre.csv")

profile_df = pd.read_csv("../results/sweeps/user_specific_profile.csv")
genre_df   = pd.read_csv("../results/sweeps/user_specific_genre.csv")
combo_df   = pd.read_csv("../results/sweeps/user_specific_combined.csv")

plt.figure(figsize=(7,5))

plt.scatter(global_df["novelty"], global_df["ndcg"], label="Global novelty", alpha=0.7)
plt.scatter(profile_df["novelty"], profile_df["ndcg"], label="Profile-based novelty", s=80)
plt.scatter(genre_df["novelty"], genre_df["ndcg"], label="Genre-diversity novelty", s=80)
plt.scatter(combo_df["novelty"], combo_df["ndcg"], label="Combined", s=80)

plt.xlabel("Novelty")
plt.ylabel("NDCG")
plt.title("Accuracy–Novelty Tradeoff (User-specific vs Global)")
plt.legend()
plt.grid(alpha=0.3)
plt.show()

### How did we group users?
<b> A) Profile Size Grouping (Interactions per User):</b>
- Users with few interactions have very uncertain distance estimates, too little history to judge novelty → low λ.
- Users with many interactions have a stable, well-defined taste profile → novelty becomes meaningful → higher λ.

- Distribution:
    - mean: 42 items
    - median: 26
    - 25% < 10
    - 50% < 26
    - 75% < 55

| Profile size | Rationale                          | Novelty weight λ |
| ------------ | ---------------------------------- | ---------------- |
| < 10 items   | sparse history, novelty unstable   | **0.05**         |
| 10–55 items  | majority of users, stable baseline | **0.20**         |
| > 55 items   | rich profiles, novelty meaningful  | **0.50**         |

<b> B) Genre Diversity Grouping (Entropy) </b>
- Entropy is a measure of how diverse or spread out the genres in a user's history are.
    - A user who only plays Action has: `entropy ≈ 0`
    - A user who plays Action + RPG + Strategy + Indie has: `entropy ≈ higher`
- Intuition:
    - Low entropy → niche taste → novelty may hurt accuracy
    - High entropy → broad taste → novelty may help
    - Medium entropy → balanced

| Genre Entropy | Meaning         | λ        |
| ------------- | --------------- | -------- |
| low           | niche taste     | **0.05** |
| medium        | mixed           | **0.20** |
| high          | very open taste | **0.50** |

<b> C) Combined Grouping </b>
- You simply take the average λ from profile-size and genre-diversity groups.
- This creates a hybrid behavior:
    - Users with large profiles and high genre entropy → highest novelty
    - Users with small profiles and low entropy → minimal novelty
    - Mixed cases → in between

### Why all user-specific results ended up so close
<b>Reason 1—The majority of users fall in the middle groups</b>

Most users have:
* profile size = 10–55
* entropy = mid-range

→ They all received λ = 0.20.

<b>Reason 2—Low and high groups were small</b>

Very few users received λ = 0.05 or λ = 0.50.

Since only 3000 users were sampled in the sweep, the differences shrink even further.

<b>Reason 3—EASE scores dominate the ranking</b>

EASE provides strong baseline scores, so novelty adjustments with moderate λ values (0.05–0.5) produce only small ranking changes.

<b>`s' = s + λ*N`</b>

### Conclusion: Effect of User-Specific Novelty Weighting
- We tested three user-specific novelty weighting strategies, based on interaction profile size, genre diversity (entropy), and a hybrid of both, and compared them to the global novelty baseline.
- Across all three strategies, the resulting novelty values clustered around 0.66, with NDCG scores between 0.214 and 0.219. These points lie in the same region as the global novelty sweep with λ ≈ 0.1–0.2, which represents the best accuracy–novelty balance for fixed global values.
- <b>The main result is that user-specific novelty weighting does not significantly outperform global novelty weighting.</b>
    - Instead, it reproduces the upper-left portion of the global accuracy–novelty trade-off curve without providing a more favorable trade-off.
- This limited improvement is likely due to:
    - Most users have similar interaction counts, causing many of them to receive the same novelty weight.
    - Genre diversity (entropy) is also being concentrated in a narrow range, again reducing differentiation between users.
    - EASE scores are dominant, so novelty adjustments with moderate λ values (0.05–0.5) produce only small ranking changes.

| Method          | Novelty | NDCG        | Comment                                     |
| --------------- | ------- | ----------- | ------------------------------------------- |
| Profile size    | 0.659   | **0.21925** | Slightly lower novelty, reasonable accuracy |
| Genre diversity | 0.6655  | 0.21489     | Close to profile, a bit worse               |
| Combined        | 0.6663  | 0.21752     | In between                                  |


# SAVE A NOVELTY-RERANKED SUBMISSION FOR CODABENCH

In [ ]:
from src.experiments.user_specific_sweep import user_specific_rerank
from src.data.loader import load_interactions, load_games
from src.novelty.user_groups import (
    assign_groups_by_profile_size,
    assign_groups_by_genre_diversity,
    assign_groups_combined,
)
import numpy as np
from src.config import LAMBDA_REG
from src.models.ease import EASE
from src.pipelines.save import save_submission

# 1. Load data
train = load_interactions(train=True)
test_in = load_interactions(train=False)
games = load_games()

item_distance = np.load("../data/processed/genre_distance.npy")

# 3. Train EASE with fixed lambda_reg
model = EASE(lambda_reg=float(LAMBDA_REG))

# recommend() already uses a Codabench style mapping
recs_grouping = model.recommend(train, test_in, top_k=300)

# user-specific lambda per user, computed from the fold in interactions (test_in)
lambda_profile  = assign_groups_by_profile_size(test_in)
lambda_genre    = assign_groups_by_genre_diversity(test_in, games)
lambda_combined = assign_groups_combined(test_in, games)

def build_and_save_user_specific_submission(recs, test_in, item_distance, user_lambda, grouping):
    recs_us = user_specific_rerank(
        recs,
        test_in,
        item_distance,
        user_lambda,
    )

    recs_us = (
        recs_us
        .sort_values(["user_id", "score"], ascending=[True, False])
        .groupby("user_id")
        .head(20)
        .reset_index(drop=True)
    )

    submission = recs_us[["user_id", "item_id"]]
    save_submission(submission, f"ease_lambda{LAMBDA_REG}_usnov_{grouping}")

# build three Codabench submissions
build_and_save_user_specific_submission(recs_grouping, test_in, item_distance, lambda_profile,  "profile-v2")
build_and_save_user_specific_submission(recs_grouping, test_in, item_distance, lambda_genre,    "genre-v2")
build_and_save_user_specific_submission(recs_grouping, test_in, item_distance, lambda_combined, "combined-v2")

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# ----- hard coded metrics -----
baseline = {
    "novelty": 0.7571858344680189,
    "ndcg":   0.24037183269526935,
}

global_points = [
    {"lambda": 0.0, "novelty": 0.7571858344680189, "ndcg": 0.24037183269526935},
    {"lambda": 0.1, "novelty": 0.7675673744963164, "ndcg": 0.234885227845009},
    {"lambda": 0.2, "novelty": 0.776181917316719,  "ndcg": 0.2269467830913513},
    {"lambda": 0.3, "novelty": 0.7829369966718916, "ndcg": 0.21763760094619242},
    {"lambda": 0.4, "novelty": 0.7885775842741768, "ndcg": 0.2089873200319885},
    {"lambda": 0.5, "novelty": 0.7933494348188609, "ndcg": 0.20011688373326467},
]

user_points = [
    {
        "label": "User profile novelty",
        "novelty": 0.7688015160867505,
        "ndcg":   0.23173826062697006,
    },
    {
        "label": "User genre novelty",
        "novelty": 0.7718773808736236,
        "ndcg":   0.22984383937047892,
    },
    {
        "label": "User combined novelty",
        "novelty": 0.7702365958206725,
        "ndcg":   0.23117905870278926,
    },
]

# ----- turn into DataFrames for convenience -----
df_global = pd.DataFrame(global_points)
df_user   = pd.DataFrame(user_points)

plt.figure(figsize=(7, 5))

# 1) baseline point
plt.scatter(
    baseline["novelty"],
    baseline["ndcg"],
    color="black",
    marker="*",
    s=140,
    label="Baseline EASE (λ = 300)",
)

# 2) global novelty sweep as a line with points
plt.plot(
    df_global["novelty"],
    df_global["ndcg"],
    marker="o",
    linestyle="-",
    color="tab:blue",
    label="Global novelty weights",
    alpha=0.5
)

# label lambda values near the global points (short labels)
for nov, nd, lam in zip(df_global["novelty"], df_global["ndcg"], df_global["lambda"]):
    plt.annotate(
        f"{lam}",
        (nov, nd),
        textcoords="offset points",
        xytext=(3, 3),
        fontsize=8,
        color="tab:blue",
    )

# 3) user specific methods as separate colored markers
plt.scatter(
    df_user["novelty"],
    df_user["ndcg"],
    color="tab:orange",
    marker="D",
    s=70,
    label="User specific groups",
)

# optional: short labels for the three user methods
for _, row in df_user.iterrows():
    short = row["label"].replace("User ", "").replace(" novelty", "")
    plt.annotate(
        short,  # "profile", "genre", "combined"
        (row["novelty"], row["ndcg"]),
        textcoords="offset points",
        xytext=(4, 2),
        fontsize=8,
        color="tab:orange",
    )

plt.xlabel("Novelty")
plt.ylabel("NDCG")
plt.title("Accuracy vs Novelty: baseline, global novelty, user specific groups")
plt.grid(alpha=0.3)
plt.legend(loc="lower left")
plt.tight_layout()
plt.show()
